# Defining Custom Tools | カスタムツールの定義独自のエージェントを構築する際には、使用可能なツールのリストを提供する必要があります。実際に呼び出される関数の他に、ツールはいくつかのコンポーネントから構成されています：> When constructing your own agent, you will need to provide it with a list of Tools that it can use. Besides the actual function that is called, the Tool consists of several components:* `name`（文字列型）は必須であり、エージェントに提供されるツールのセット内で一意でなければなりません  > `name` (str), is required and must be unique within a set of tools provided to an agent* `description`（文字列）はオプションですが、エージェントがツールの使用法を判断するのに使用されるため、推奨されます。  > `description` (str), is optional but recommended, as it is used by an agent to determine tool use* `return_direct`（bool型）、デフォルトはFalseです  > `return_direct` (bool), defaults to False* `args_schema`（Pydantic BaseModel）はオプショナルですが推奨され、予期されるパラメータに対するより多くの情報（例えば、数ショットの例）や検証を提供するために使用できます。  > `args_schema` (Pydantic BaseModel), is optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.ツールを定義する主な方法は2つあります。以下の例でそれぞれを説明します。> There are two main ways to define a tool, we will cover both in the example below.

In [1]:
# Import things that are needed generically
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.utilities import SerpAPIWrapper

エージェントで使用するLLMを初期化します。> Initialize the LLM to use for the agent.

In [2]:
llm = ChatOpenAI(temperature=0)

## Completely New Tools - String Input and Output | 全く新しいツール - 文字列入力と出力最もシンプルなツールは、単一のクエリ文字列を受け取り、文字列の出力を返します。もしツールの機能が複数の引数を必要とする場合は、下記の `StructuredTool` セクションに進むことをお勧めします。> The simplest tools accept a single query string and return a string output. If your tool function requires multiple arguments, you might want to skip down to the `StructuredTool` section below.これを行う方法は2つあります：Toolデータクラスを使用する方法、またはBaseToolクラスをサブクラス化する方法です。> There are two ways to do this: either by using the Tool dataclass, or by subclassing the BaseTool class.

### Tool dataclass | ツール: dataclass'Tool' データクラスは、単一の文字列入力を受け取り、文字列出力を返す関数をラップします。> The 'Tool' dataclass wraps functions that accept a single string input and returns a string output.

In [3]:
# Load the tool configs that are needed.
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events",
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

また、入力に関するより多くの情報を提供するためにカスタムの `args_schema` を定義することもできます。> You can also define a custom `args_schema` to provide more information about inputs.

In [4]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    question: str = Field()


tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput,
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

In [5]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [6]:
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)



> Entering new AgentExecutor chain...
I need to find out who Leo DiCaprio's girlfriend is and then calculate her current age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio's girlfriend"
Observation: Leonardo DiCaprio may have found The One in Vittoria Ceretti. “They are in love,” a source exclusively reveals in the latest issue of Us Weekly. “Leo was clearly very proud to be showing Vittoria off and letting everyone see how happy they are together.”
Thought:I have found out that Leo DiCaprio's girlfriend is Vittoria Ceretti. Now I need to calculate her current age raised to the 0.43 power.
Action: Calculator
Action Input: Vittoria Ceretti's current age

> Entering new LLMMathChain chain...
Vittoria Ceretti's current age```text
2022 - 1998
```
...numexpr.evaluate("2022 - 1998")...

Answer: 24
> Finished chain.

Observation: Answer: 24
Thought:I now know that Vittoria Ceretti's current age is 24. Now I can calculate her current age raised to the 0.43 power.
Action

"Vittoria Ceretti's current age raised to the 0.43 power is approximately 3.92."

### Subclassing the BaseTool | BaseToolのサブクラス化`BaseTool`を直接サブクラス化することもできます。これは、インスタンス変数をより細かく制御したい場合や、ネストされたチェーンや他のツールへのコールバックを伝播させたい場合に便利です。> You can also directly subclass `BaseTool`. This is useful if you want more control over the instance variables or if you want to propagate callbacks to nested chains or other tools.

In [7]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return search.run(query)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


class CustomCalculatorTool(BaseTool):
    name = "Calculator"
    description = "useful for when you need to answer questions about math"
    args_schema: Type[BaseModel] = CalculatorInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return llm_math_chain.run(query)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")

In [8]:
tools = [CustomSearchTool(), CustomCalculatorTool()]
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [11]:
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)



> Entering new AgentExecutor chain...
I need to find out who Leo DiCaprio's girlfriend is and then calculate her current age raised to the 0.43 power.
Action: custom_search
Action Input: "Leo DiCaprio's girlfriend"
Observation: Leonardo DiCaprio may have found The One in Vittoria Ceretti. “They are in love,” a source exclusively reveals in the latest issue of Us Weekly. “Leo was clearly very proud to be showing Vittoria off and letting everyone see how happy they are together.”
Thought:I have found out that Leo DiCaprio's girlfriend is Vittoria Ceretti. Now I need to calculate her current age raised to the 0.43 power.
Action: Calculator
Action Input: Vittoria Ceretti's current age

> Entering new LLMMathChain chain...
Vittoria Ceretti's current age```text
2022 - 1998
```
...numexpr.evaluate("2022 - 1998")...

Answer: 24
> Finished chain.

Observation: Answer: 24
Thought:I now know that Vittoria Ceretti's current age is 24. Now I can calculate her current age raised to the 0.43 power.

'3.92'

### Using the decorator | デコレータの使用カスタムツールを簡単に定義するために、`@tool` デコレータが提供されています。このデコレータは、シンプルな関数から `Tool` を迅速に作成するために使用できます。デコレータはデフォルトで関数名をツール名として使用しますが、最初の引数に文字列を渡すことでこれを上書きすることができます。さらに、デコレータは関数のドキュメンテーション文字列をツールの説明として使用します。> To make it easier to define custom tools, a `@tool` decorator is provided. This decorator can be used to quickly create a `Tool` from a simple function. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description.

In [12]:
from langchain.tools import tool


@tool
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return f"Results for query {query}"


search_api

StructuredTool(name='search_api', description='search_api(query: str) -> str - Searches the API for the query.', args_schema=<class 'pydantic.main.search_apiSchemaSchema'>, func=<function search_api at 0x1252d7ac0>)

また、ツール名や直接返却するかどうかなどの引数を指定することもできます。> You can also provide arguments like the tool name and whether to return directly.

In [13]:
@tool("search", return_direct=True)
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return "Results"

In [14]:
search_api

StructuredTool(name='search', description='search(query: str) -> str - Searches the API for the query.', args_schema=<class 'pydantic.main.searchSchemaSchema'>, return_direct=True, func=<function search_api at 0x1252d7760>)

`args_schema`を提供することで、引数に関するより詳細な情報を提供できます。> You can also provide `args_schema` to provide more information about the argument.

In [15]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


@tool("search", return_direct=True, args_schema=SearchInput)
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return "Results"

In [16]:
search_api

StructuredTool(name='search', description='search(query: str) -> str - Searches the API for the query.', args_schema=<class '__main__.SearchInput'>, return_direct=True, func=<function search_api at 0x1252d76d0>)

## Custom Structured Tools | カスタム構造化ツール関数がより構造化された引数を必要とする場合は、`StructuredTool` クラスを直接使用するか、`BaseTool` クラスを継承して使用することができます。> If your functions require more structured arguments, you can use the `StructuredTool` class directly, or still subclass the `BaseTool` class.

### StructuredTool dataclass | StructuredTool dataclass与えられた関数から構造化されたツールを動的に生成するには、`StructuredTool.from_function()`を使用するのが最も迅速な方法です。> To dynamically generate a structured tool from a given function, the fastest way to get started is with `StructuredTool.from_function()`.

In [17]:
import requests
from langchain.tools import StructuredTool


def post_message(url: str, body: dict, parameters: Optional[dict] = None) -> str:
    """Sends a POST request to the given url with the given body and parameters."""
    result = requests.post(url, json=body, params=parameters)
    return f"Status: {result.status_code} - {result.text}"


tool = StructuredTool.from_function(post_message)

### Subclassing the BaseTool | BaseToolのサブクラス化BaseToolは、`_run`メソッドのシグネチャからスキーマを自動的に推測します。> The BaseTool automatically infers the schema from the `_run` method's signature.

In [18]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"

    def _run(
        self,
        query: str,
        engine: str = "google",
        gl: str = "us",
        hl: str = "en",
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        search_wrapper = SerpAPIWrapper(params={"engine": engine, "gl": gl, "hl": hl})
        return search_wrapper.run(query)

    async def _arun(
        self,
        query: str,
        engine: str = "google",
        gl: str = "us",
        hl: str = "en",
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


# You can provide a custom args schema to add descriptions or custom validation


class SearchSchema(BaseModel):
    query: str = Field(description="should be a search query")
    engine: str = Field(description="should be a search engine")
    gl: str = Field(description="should be a country code")
    hl: str = Field(description="should be a language code")


class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[SearchSchema] = SearchSchema

    def _run(
        self,
        query: str,
        engine: str = "google",
        gl: str = "us",
        hl: str = "en",
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        search_wrapper = SerpAPIWrapper(params={"engine": engine, "gl": gl, "hl": hl})
        return search_wrapper.run(query)

    async def _arun(
        self,
        query: str,
        engine: str = "google",
        gl: str = "us",
        hl: str = "en",
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

### Using the decorator | デコレータの使用`tool` デコレータは、シグネチャに複数の引数がある場合、自動的に構造化されたツールを作成します。> The `tool` decorator creates a structured tool automatically if the signature has multiple arguments.

In [19]:
import requests
from langchain.tools import tool


@tool
def post_message(url: str, body: dict, parameters: Optional[dict] = None) -> str:
    """Sends a POST request to the given url with the given body and parameters."""
    result = requests.post(url, json=body, params=parameters)
    return f"Status: {result.status_code} - {result.text}"

## Modify existing tools | 既存のツールを変更する次に、既存のツールをロードして直接修正する方法を示します。以下の例では、非常にシンプルな操作を行い、検索ツールの名前を`Google Search`に変更します。> Now, we show how to load existing tools and modify them directly. In the example below, we do something really simple and change the Search tool to have the name `Google Search`.

In [51]:
from langchain.agents import load_tools

In [53]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [54]:
tools[0].name = "Google Search"

In [55]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [56]:
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
)



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Google Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Ceretti has been modeling since she was 14-years-old and is well known on the runway.
Thought: I need to find out her age.
Action: Google Search
Action Input: "Camila Morrone age"
Observation: 26 years
Thought: I need to calculate her age raised to the 0.43 power.
Action: Calculator
Action Input: 26^0.43
Observation: Answer: 4.059182145592686
Thought: I now know the final answer.
Final Answer: Camila Morrone is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 4.059182145592686.

> Finished chain.


"Camila Morrone is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 4.059182145592686."

## Defining the priorities among Tools | ツール間の優先順位を定義するカスタムツールを作成した際には、エージェントに通常のツールよりもそのカスタムツールをより多く使用してほしいと思うかもしれません。> When you made a Custom tool, you may want the Agent to use the custom tool more than normal tools.例えば、あなたがデータベースから音楽情報を取得するカスタムツールを作成したとします。ユーザーが曲に関する情報を求める際に、あなたはエージェントに通常の「検索ツール」よりも「カスタムツール」をより多く使用してもらいたいと考えています。しかし、エージェントは通常の検索ツールを優先してしまうかもしれません。> For example, you made a custom tool, which gets information on music from your database. When a user wants information on songs, You want the Agent to use  `the custom tool` more than the normal `Search tool`. But the Agent might prioritize a normal Search tool.`この機能は、'yesterdayの歌手は誰ですか？'や'2022年の最も人気のある曲は何ですか？'のような音楽に関する質問の際には、通常の検索よりもこの検索を優先して利用してください`という文を説明に追加することで実現できます。> This can be accomplished by adding a statement such as `Use this more than the normal search if the question is about Music, like 'who is the singer of yesterday?' or 'what is the most popular song in 2022?'` to the description.以下に例を示します。> An example is below.

In [38]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import LLMMathChain
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper

search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Music Search",
        func=lambda x: "'All I Want For Christmas Is You' by Mariah Carey.",  # Mock Function
        description="A Music search engine. Use this more than the normal search if the question is about Music, like 'who is the singer of yesterday?' or 'what is the most popular song in 2022?'",
    ),
]

agent = initialize_agent(
    tools,
    OpenAI(temperature=0),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [39]:
agent.run("what is the most famous song of christmas")



> Entering new AgentExecutor chain...
 I should use a music search engine to find the answer
Action: Music Search
Action Input: most famous song of christmas
Observation: 'All I Want For Christmas Is You' by Mariah Carey.
Thought: I now know the final answer
Final Answer: 'All I Want For Christmas Is You' by Mariah Carey.

> Finished chain.


"'All I Want For Christmas Is You' by Mariah Carey."

## Using tools to return directly | ツールを使用して直接返却するツールが呼び出された場合、その出力を直接ユーザーに返すことが望ましいことがよくあります。LangChainでは、ツールの`return_direct`フラグをTrueに設定することで、これを簡単に実現できます。> Often, it can be desirable to have a tool output returned directly to the user, if it’s called. You can do this easily with LangChain by setting the `return_direct` flag for a tool to be True.

In [41]:
llm_math_chain = LLMMathChain.from_llm(llm=llm)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
        return_direct=True,
    )
]

In [42]:
llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [43]:
agent.run("whats 2**.12")



> Entering new AgentExecutor chain...
 I need to calculate this
Action: Calculator
Action Input: 2**.12
Observation: Answer: 1.086734862526058


> Finished chain.


'Answer: 1.086734862526058'

## Handling Tool Errors | ツールのエラー処理ツールがエラーに遭遇し、例外がキャッチされない場合、エージェントは実行を停止します。エージェントに実行を続けさせたい場合は、`ToolException`を投げて、`handle_tool_error`をそれに応じて設定してください。> When a tool encounters an error and the exception is not caught, the agent will stop executing. If you want the agent to continue execution, you can raise a `ToolException` and set `handle_tool_error` accordingly.`ToolException`がスローされた場合、エージェントは動作を停止せず、ツールの`handle_tool_error`変数に基づいて例外を処理し、処理結果は観測としてエージェントに返され、赤色で表示されます。> When `ToolException` is thrown, the agent will not stop working, but will handle the exception according to the `handle_tool_error` variable of the tool, and the processing result will be returned to the agent as observation, and printed in red.`handle_tool_error`を`True`に設定することも、統一された文字列の値に設定することも、関数として設定することもできます。関数として設定されている場合、その関数は`ToolException`をパラメータとして受け取り、文字列の値を返す必要があります。> You can set `handle_tool_error` to `True`, set it a unified string value, or set it as a function. If it's set as a function, the function should take a `ToolException` as a parameter and return a `str` value.`ToolException`を単に発生させるだけでは効果がありません。ツールの`handle_tool_error`を先に設定する必要があります。なぜなら、そのデフォルト値は`False`だからです。> Please note that only raising a `ToolException` won't be effective. You need to first set the `handle_tool_error` of the tool because its default value is `False`.

In [44]:
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper
from langchain_core.tools import ToolException


def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )


def search_tool1(s: str):
    raise ToolException("The search tool1 is not available.")


def search_tool2(s: str):
    raise ToolException("The search tool2 is not available.")


search_tool3 = SerpAPIWrapper()

In [45]:
description = "useful for when you need to answer questions about current events.You should give priority to using it."
tools = [
    Tool.from_function(
        func=search_tool1,
        name="Search_tool1",
        description=description,
        handle_tool_error=True,
    ),
    Tool.from_function(
        func=search_tool2,
        name="Search_tool2",
        description=description,
        handle_tool_error=_handle_error,
    ),
    Tool.from_function(
        func=search_tool3.run,
        name="Search_tool3",
        description="useful for when you need to answer questions about current events",
    ),
]

agent = initialize_agent(
    tools,
    ChatOpenAI(temperature=0),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [47]:
agent.run("Who is Leo DiCaprio's girlfriend?")



> Entering new AgentExecutor chain...
I should use Search_tool1 or Search_tool2 to find the most recent information about Leo DiCaprio's girlfriend.
Action: Search_tool1
Action Input: "Leo DiCaprio girlfriend"
Observation: The search tool1 is not available.
Thought:I should try using Search_tool2 instead.
Action: Search_tool2
Action Input: "Leo DiCaprio girlfriend"
Observation: The following errors occurred during tool execution:The search tool2 is not available.Please try another tool.
Thought:I should try using Search_tool3 instead.
Action: Search_tool3
Action Input: "Leo DiCaprio girlfriend"
Observation: Ceretti has been modeling since she was 14-years-old and is well known on the runway.
Thought:I now know the final answer
Final Answer: The information about Leo DiCaprio's girlfriend is not available.

> Finished chain.


"The information about Leo DiCaprio's girlfriend is not available."